In [2]:
from google.protobuf.internal.decoder import _DecodeVarint32

from proto.training_dataset_pb2 import DictionaryEntry

class Dictionary(object):
    def __init__(self):
        with open('/home/klopyrev/squad/dataset/featuredictbucketized.proto', 'rb') as fileobj:
            data = fileobj.read()
            self._msgs = []
            pos = 0
            while pos < len(data):
                msg_length, msg_pos = _DecodeVarint32(data, pos)
                self._msgs.append(data[msg_pos:msg_pos + msg_length])
                pos = msg_pos + msg_length

    def GetName(self, index):
        msg = self._msgs[index]
        entry = DictionaryEntry()
        entry.ParseFromString(msg)
        return entry.name

dictionary = Dictionary()

In [3]:
from proto.io import ReadArticles
from proto.training_dataset_pb2 import TrainingArticle

dev_articles = dict(map(lambda article: (article.title, article), ReadArticles('/home/klopyrev/squad/dataset/test-annotatedpartial.proto')))
dev_features = dict(map(lambda article: (article.title, article), ReadArticles('/home/klopyrev/squad/dataset/test-featuresbucketized.proto', cls=TrainingArticle)))

In [4]:
import random

questions = []
for title, article in dev_articles.iteritems():
    for paragraph_index, paragraph in enumerate(article.paragraphs):
        for question_index in xrange(len(paragraph.qas)):
            if title == 'Precipitation':
                questions.append((title, paragraph_index, question_index))
random.shuffle(questions)

In [5]:
def GetCorrectSpanAndFeatures(title, paragraph_id, question_id):
    article = dev_features[title]
    paragraph = article.paragraphs[paragraph_id]
    question = paragraph.questions[question_id]
    span = paragraph.candidateAnswers[question.correctAnswerIndex]
    features = map(lambda index: dictionary.GetName(index), question.candidateAnswerFeatures[question.correctAnswerIndex].indices)
    return span, features

In [7]:
for i in range(0, len(questions)):
    title, paragraph_index, question_index = questions[i]
    print title
    
    paragraph = dev_articles[title].paragraphs[paragraph_index]
    question = paragraph.qas[question_index]
    span, features = GetCorrectSpanAndFeatures(title, paragraph_index, question_index)
    tokens = paragraph.context.sentence[span.sentenceIndex].token
    print question.question.text
    print ''.join([token.word + token.after for token in tokens])
    print question.answers[0].text
    for feature in features:
        print '  ', feature
    
    print
    print
    print

Precipitation
What is another term for showery precipitation?
Convective rain, or showery precipitation, occurs from convective clouds, e.g., cumulonimbus or cumulus congestus. 
Convective rain
   Left Length = Bucket 0 of 10: 0 - 3
   Unigram QA Match TF IF (Left of Span) = Bucket 0 of 10: 0 - 0.0451953
   Unigram QA Match TF IF (Span) = Bucket 0 of 10: 0 - 0.0102663
   Bigram QA Match TF IF (Left of Span) = Bucket 1 of 10: 0 - 2.75503
   Bigram QA Match TF IF (Span) = Bucket 1 of 10: 0 - 2.44176
   Span Length = Bucket 1 of 10: 2 - 3
   ParTreeLab = NP
   Wh+ParTreeLab = WP what NP
   Unigram TF IF (Span) = Bucket 7 of 10: 12.9927 - 19.47
   AllPos = JJ NN
   Wh+AllPos = WP what JJ NN
   Unigram QA Match TF IF (Whole Sentence) = Bucket 6 of 10: 12.4896 - 16.1277
   Right Length = Bucket 5 of 10: 14 - 18
   QL+SL+DPa = what or<-cc-
   QL+SL+DPa = be or<-cc-
   QL+SL+DPa = ? or<-cc-
   Sentence Length = Bucket 1 of 10: 16 - 21
   QL+SL+DPa = ? ,<-punct-
   QL+SL+DPa = what ,<-punct-
  